## Parameters

In [ ]:
prefix='./results/'
n_jobs = 10
n_trees = 10

## Imports

In [ ]:
from metrics import *
%run ___imports.ipynb 

## Loading

In [ ]:
with open('simplified_reccurent_CD_pt1.pkl', "rb") as fh:
            df1 = pickle.load(fh)
with open('simplified_reccurent_CD_pt2.pkl', "rb") as fh:
            df2 = pickle.load(fh)        
with open('simplified_reccurent_CD_pt3.pkl', "rb") as fh:
            df3 = pickle.load(fh)       
with open('simplified_reccurent_CD_pt4.pkl', "rb") as fh:
            df4 = pickle.load(fh)

df = pd.concat([df1,df2,df3,df4])

df.TX_DATETIME = (df.TX_DATETIME - datetime.strptime('2018-04-01', '%Y-%m-%d')).dt.days

train = df[df['TX_DATETIME']>=0]
train = train[train['TX_DATETIME']<7]
test = df[df['TX_DATETIME']>=7]

## Preparation

In [ ]:
performancesMeasures = pd.DataFrame(columns=['model','day_test','pk100','pk300','pk1000'])

## Execution

In [ ]:
first_test_day = test['TX_DATETIME'].min()
last_test_day = test['TX_DATETIME'].max()
num_test_days = (last_test_day - first_test_day)
range_test_days = range(first_test_day, last_test_day + 1)

In [ ]:
important_features = train.columns.values.tolist()
to_remove = ['TX_DATETIME','TX_TIME_SECONDS','CUSTOMER_ID', 'TX_FRAUD']
for i in to_remove: important_features.remove(i)

In [ ]:
clf = BalancedRandomForestClassifier(n_jobs=n_jobs,n_estimators=n_trees,random_state=42, verbose=5)
clf.fit(train[important_features], train.TX_FRAUD)

In [ ]:
#measuring performances
for test_day in range_test_days:
    
    print("Iteration",test_day,"of",last_test_day)
    
    test_set = test[test['TX_DATETIME'] == test_day]
    last_test_set = test[test['TX_DATETIME'] == test_day - 1] #for adding fraudulent cards
    
    if test_day == first_test_day:
        fraudulent_card_train = np.unique(train.loc[train.TX_FRAUD == 1]['CUSTOMER_ID'].values)
    else:
        fraudulent_card_train = np.unique(np.concatenate([fraudulent_card_train,np.unique(last_test_set.loc[last_test_set['TX_FRAUD'] == 1]['CUSTOMER_ID'].values)]))
    
    
    daily_predi=clf.predict_proba(test_set[important_features])[:,1]

    daily_truth = test_set['TX_FRAUD']
    test_cards_ids = test_set['CUSTOMER_ID']
    
    pk100,pk300,pk1000 = compute_metrics_remove_cards(daily_truth,
                                                        daily_predi,
                                                        test_cards_ids,
                                                        fraudulent_card_train)
    model_name = prefix+'baseline_BRF'
    
    performancesMeasures.loc[len(performancesMeasures)] = [model_name,
                                                           test_day,
                                                           pk100,
                                                           pk300,
                                                           pk1000]
                                                           

In [ ]:
perf_name = prefix+'metrics_BRF_baseline.csv'
performancesMeasures.to_csv(perf_name,index=False)
performancesMeasures

In [ ]:
performancesMeasures.pk100.plot()